In [1]:
import sqlparse

from cast.cache import PostData

from django.test.client import RequestFactory
from django.db import connection, reset_queries

# Measure PostData Cache Performance

Find out which pages generate queries.

In [13]:
def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)
        

def blocker(*args):
    raise Exception("No database access allowed here.")

In [14]:
blog_slug = "ephes_blog"
# blog_slug = "das_claas_und_nora_blog"
blog = Blog.objects.get(slug=blog_slug)

In [6]:
%%time
reset_queries()
post_queryset = Post.objects.live().descendant_of(blog).order_by("-visible_date")
request_factory = RequestFactory()
post_data = PostData.create_from_post_queryset(
    request=request_factory.get("/"),
    blog=blog,
    post_queryset=post_queryset,
    template_base_dir="bootstrap4",
)
print(len(connection.queries))

bulk_to_python for cast image:  [2255]
bulk_to_python for cast image:  [2144]
bulk_to_python for cast image:  [2132]
bulk_to_python for cast image:  [1429]
bulk_to_python for cast image:  [1428]
bulk_to_python for cast image:  [1427]
bulk_to_python for cast image:  [1426]
bulk_to_python for cast image:  [1425, 1424]
bulk_to_python for cast image:  [1419]
bulk_to_python for cast image:  [1387]
in video bulk to python:  [144]
bulk_to_python for cast image:  [1376]
in video bulk to python:  [143]
in video bulk to python:  [142]
in video bulk to python:  [141]
bulk_to_python for cast image:  [1321]
bulk_to_python for cast image:  [5]
bulk_to_python for cast image:  [1282]
bulk_to_python for cast image:  [1141]
bulk_to_python for cast image:  [1140]
bulk_to_python for cast image:  [1139]
bulk_to_python for cast image:  [1136, 1138]
bulk_to_python for cast image:  [1134, 1135, 1137]
bulk_to_python for cast image:  [1131, 1124, 1118]
bulk_to_python for cast image:  [1115]
in video bulk to pyt

In [12]:
post = Post.objects.get(pk=501)

<Post: foobar>

In [11]:
%%time
request = request_factory.get("/foobar/")
posts = list(post_queryset)
reset_queries()
# request = request_factory.get(post_data.page_url_by_id[post.pk])
for post in posts:
    print("post pk: ", post.pk)
    with connection.execute_wrapper(blocker):
        response = post.serve(request, post_data=post_data).render()

post pk:  501


Exception: No database access allowed here.

In [8]:
print(len(posts))

169


In [9]:
print(len(connection.queries))

170


In [10]:
show_queries(connection.queries)

SELECT "wagtailcore_site"."id",
       "wagtailcore_site"."hostname",
       "wagtailcore_site"."port",
       "wagtailcore_site"."site_name",
       "wagtailcore_site"."root_page_id",
       "wagtailcore_site"."is_default_site",
       CASE
           WHEN ("wagtailcore_site"."hostname" = 'testserver'
                 AND "wagtailcore_site"."port" = 80) THEN 0
           WHEN ("wagtailcore_site"."hostname" = 'testserver'
                 AND "wagtailcore_site"."is_default_site") THEN 1
           WHEN "wagtailcore_site"."is_default_site" THEN 2
           ELSE 3
       END AS "match",
       "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
 

In [11]:
Post.objects.get(pk=335)

<Post: Bei Norman in Hattgenstein>

In [12]:
FluentComment.objects.count()

4